# Assignment 1 (Part 1): Multi Instant Learning

a) Pick an appropriate problem domain for multi instance learning for which a small  data set is readily available, Implement and submit a proper colab with proper ablation study (with various tunings) using tensorboard  for multi instance learning. Submit a nice readme explaining the problem you are trying to solve with multi instance learning, the approach. Make sure the colab is highly commented and readable with proper results confusion matrices, metrics, showing error examples the model not able to identify etc.,. - follow the template in https://github.com/fastai/fastbook (Links to an external site.) and https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb (Links to an external site.) and https://github.com/fastai/fastbook/blob/master/05_pet_breeds.ipynb (Links to an external site.) for the results. The results visualization is very important aspect of the overall problem.  Aim for state of art approach (extra points)  to solve this problem 

 

Note: pick reasonable training set and reasonable problem - not 100s of millions of instances.

 

Sample code/research for inspiration (dont cut and paste)  : https://medium.com/swlh/multiple-instance-learning-c49bd21f5620 (Links to an external site.)

Resources : https://learning.oreilly.com/library/view/multimodal-learning-toward/9781681736297/#toc (Links to an external site.)

 

Sample chapter for details : https://learning.oreilly.com/library/view/deep-learning-for/9780128104095/B9780128104088000067.xhtml (Links to an external site.)

 

https://github.com/AMLab-Amsterdam/AttentionDeepMIL (Links to an external site.)

https://paperswithcode.com/task/multiple-instance-learning (Links to an external site.)

https://medium.com/center-for-data-science/approaching-sound-event-detection-as-a-multiple-instance-learning-problem-3165abdfc7cc (Links to an external site.)

https://www.sciencedirect.com/science/article/abs/pii/S0167865519302971

In [1]:
# Load required libraries
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from torchvision.models import resnet
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.models.resnet import ResNet, BasicBlock
from torch.utils.data.dataset import Dataset
from sklearn import model_selection as ms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from functools import partial, reduce
from torch.nn import functional as F
from tqdm.autonotebook import tqdm
from sklearn import metrics as mtx
from random import shuffle
import inspect
import time
from torch import nn, optim
import os
import time
import datetime
import copy
import re
import yaml
import uuid
import warnings
import random
import pickle

In [2]:
# Pretrain the model
def get_data(train_batch_size, val_batch_size):
    #Download the MNIST dataset
    data_mnist = MNIST(download=True, train=True, root=".").train_data.float()
    # Normalize and resize the images
    data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((data_mnist.mean()/255,), (data_mnist.std()/255,))])
    # Load the training dataset
    train_data = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)
    #Load the validation dataset
    val_data = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=val_batch_size, shuffle=False)
    return train_data, val_data

In [3]:
train_batch_size = 100
val_batch_size = 100

train_data, valid_data = get_data(train_batch_size, val_batch_size)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [4]:
#Define the model
class Mnist_resnet(ResNet):
    def __init__(self):
        super(Mnist_resnet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(Mnist_resnet, self).forward(x), dim=-1)

In [5]:
# Helper function
def metric_calculation(metric, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric).args:
        return metric(true_y, pred_y, average="macro")
    else:
        return metric(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [6]:
start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Model
model = Mnist_resnet().to(device)

# Parameter specification
epochs = 3
train_data, val_data = get_data(train_batch_size, val_batch_size)

# Loss function
loss_function = nn.CrossEntropyLoss()         

# Optimizer
optimizer = optim.Adadelta(model.parameters())


losses = []
train_batches = len(train_data)
val_batches = len(val_data)

for epoch in range(epochs):
    total_loss = 0

    # display progress bar 
    progress = tqdm(enumerate(train_data), desc="Loss: ", total=train_batches)

    # train the model
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        model.zero_grad() 
        # forward propagation
        output = model(X)
        # Get the loss
        loss = loss_function(output, y)
        # Backward propagation
        loss.backward()
        # Optimize the loss and perform parameter update as per current gradient
        optimizer.step() 

        # get the training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # display updated progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Validate the model
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_data):
            X, y = data[0].to(device), data[1].to(device)

            output= model(X)

            val_losses += loss_function(output, y)
            # Get Prediction
            predicted_classes = torch.max(output, 1)[1]
            
            # calculate  various metrics for a batch
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(metric_calculation(metric, y.cpu(), predicted_classes.cpu()))
          
    print(f"Epoch: {epoch+1}/{epochs}, Training loss: {total_loss/train_batches}, Validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/train_batches)


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")



Epoch: 1/3, Training loss: 1.556356259584427, Validation loss: 1.4891477823257446
	     precision: 0.9762
	        recall: 0.9746
	            F1: 0.9726
	      accuracy: 0.9747



Epoch: 2/3, Training loss: 1.4764532792568206, Validation loss: 1.4758780002593994
	     precision: 0.9868
	        recall: 0.9865
	            F1: 0.9855
	      accuracy: 0.9868



Epoch: 3/3, Training loss: 1.4720717054605483, Validation loss: 1.4857134819030762
	     precision: 0.9797
	        recall: 0.9778
	            F1: 0.9769
	      accuracy: 0.9782


In [7]:
#Save the model
torch.save(model.state_dict(), 'mnist_model.pt')

# Data Generation for Multi-Instance Learning

In [8]:
# Load the data for data generation
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [9]:
x_train = x_train[:30001]
y_train = y_train[:30001]
x_test = x_test[:9000]
y_test = y_test[:9000]

In [10]:
# Change the datatype to Float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
x_train = x_train / 255
x_test = x_test / 255
print('Shape of x_train:', x_train.shape)
print('Total number of images in x_train:', x_train.shape[0])
print('Total number of images in x_test:', x_test.shape[0])

Shape of x_train: (30001, 28, 28)
Total number of images in x_train: 30001
Total number of images in x_test: 9000


Create tuple for train and test

In [11]:
instance_index_label_train = [(i, y_train[i]) for i in range(x_train.shape[0])]
instance_index_label_test = [(i, y_test[i]) for i in range(x_test.shape[0])]

In [12]:
# find the index if label is 1 in x_train labels
find_index_train = [instance_index_label_train[i][0] for i in range(len(instance_index_label_train)) if instance_index_label_train[i][1]==1]
# find the index if label is 1 in x_test labels
find_index_test = [instance_index_label_test[i][0] for i in range(len(instance_index_label_test)) if instance_index_label_test[i][1]==1]

In [13]:
print('X_train Index:', instance_index_label_train[0][0])
print('X_train Label:', instance_index_label_train[0][1])

X_train Index: 0
X_train Label: 5


Load the pretrained model

In [14]:

class Mnist_ResNet(ResNet):
    def __init__(self):
        super(Mnist_ResNet, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(Mnist_ResNet, self).forward(x), dim=-1)

In [15]:
model = Mnist_ResNet()
model.load_state_dict(torch.load('mnist_model.pt'))
body = nn.Sequential(*list(model.children()))
# extraction of the last layer
model = body[:9]
# Evaluate the model
model.eval()

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [16]:
#Get Features

train_batch_size = 1
val_batch_size = 1
train_data, val_data = get_data(train_batch_size, val_batch_size)
#Use crossentropy loss function which is suitable for Multi Class Learning
loss_function = nn.CrossEntropyLoss()
# Optimizer for updating parameters
optimizer = optim.Adadelta(model.parameters())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [17]:
losses = []
train_batches = len(train_data)
val_batches = len(val_data)

Get features for train

In [18]:
meta_table = dict()
feature_result = []

# display progress
p = tqdm(enumerate(train_data), desc="Loss: ", total=train_batches)
# Evaluate model
model.eval()

for i, data in p:
    if i==30001:
        break
    X, y = data[0], data[1]
    model.zero_grad()
    output = model(X)
    feature_result.append(output.reshape(-1).tolist())
    meta_table[i] = output.reshape(-1).tolist()
    
feature_array = np.array(feature_result)
np.save('feature_array_full',feature_array )


In [19]:
# load the features
feature_array = np.load('feature_array_full.npy', allow_pickle=True)

Get features for test

In [20]:

# loop for every epoch (training + evaluation)
meta_test_table = dict()
feature_test_result = []

# progress bar
progress = tqdm(enumerate(val_data), desc="Loss: ", total=val_batches)

model.eval()

for i, data in progress:
    if i==9000:
        break
    X, y = data[0], data[1]
    # training step for single batch
    model.zero_grad()
    output_val = model(X)
    feature_test_result.append(output_val.reshape(-1).tolist())
    meta_test_table[i] = output_val.reshape(-1).tolist()

feature_test_array = np.array(feature_test_result)
np.save('feature_test_array_full',feature_test_array )

In [21]:
#load validation features
feature_test_array = np.load('feature_test_array_full.npy', allow_pickle=True)

Generate data for train

In [22]:

from typing import List, Dict, Tuple
def generate_data(index_label: List[Tuple]) -> List[Dict]:
    bag_size = np.random.randint(3,7,size=len(index_label)//5)
    data_copy = copy.copy(index_label)
    np.random.shuffle(data_copy)
    bags = {}
    bags_per_instance_labels = {}
    bags_labels = {}
    for bag_ind, size in enumerate(bag_size):
        bags[bag_ind] = []
        bags_per_instance_labels[bag_ind] = []
        try:
            for _ in range(size):
                inst_ind, lbl = data_copy.pop()
                bags[bag_ind].append(inst_ind)
                bags_per_instance_labels[bag_ind].append(lbl)
            bags_labels[bag_ind] = bag_label_from_instance_labels(bags_per_instance_labels[bag_ind])
        except:
            break
    return bags, bags_labels

def bag_label_from_labels(instance_labels):
    return int(any(((x==1) for x in instance_labels)))

In [23]:

bag_indices, bag_labels = generate_data(instance_index_label_train)
bag_features = {kk: torch.Tensor(feature_array[inds]) for kk, inds in bag_indices.items()}

In [24]:
# save & load bags for train data
import pickle
pickle.dump(bag_indices, open( "bag_indices", "wb" ) )
pickle.dump(bag_labels, open( "bag_labels", "wb" ) )
pickle.dump(bag_features, open( "bag_features", "wb" ) )


bag_indices = pickle.load( open( "bag_indices", "rb" ) )
bag_labels = pickle.load( open( "bag_labels", "rb" ) )
bag_features = pickle.load( open( "bag_features", "rb" ) )

Generate data for test

In [25]:
bag_test_indices, bag_test_labels = generate_data(instance_index_label_test)

In [26]:
bag_test_features = {kk: torch.Tensor(feature_test_array[inds]) for kk, inds in bag_test_indices.items()}

In [27]:
# Save and load bags for test data
pickle.dump(bag_test_indices, open( "bag_test_indices", "wb" ) )
pickle.dump(bag_test_labels, open( "bag_test_labels", "wb" ) )
pickle.dump(bag_test_features, open( "bag_test_features", "wb" ) )

bag_test_indices = pickle.load( open( "bag_test_indices", "rb" ) )
bag_test_labels = pickle.load( open( "bag_test_labels", "rb" ) )
bag_test_features = pickle.load( open( "bag_test_features", "rb" ) )

#Multiple Instance Learning

Data Preparation

In [28]:

class Data_transformation(Dataset):

    def __init__(self, data, transform=None):
        self.transform = transform
        self.data = data
        
    def __getitem__(self, index):
        tensor = self.data[index][0]
        if self.transform is not None:
            tensor = self.transform(tensor)
        return (tensor, self.data[index][1])

    def __len__(self):
        return len(self.data)

In [38]:
for i in range(len(bag_features)):
  try:
    train_data = (bag_features[i],bag_labels[i])
  except KeyError:
    continue


In [43]:
bag_features

{0: tensor([[0.1919, 0.4568, 3.2667,  ..., 0.3114, 2.4619, 0.0102],
         [1.2604, 1.7981, 0.1783,  ..., 0.2841, 0.2446, 1.8779],
         [1.3300, 2.0939, 0.1710,  ..., 0.7000, 0.3127, 2.0590]])}

In [30]:
bag_features[0]

tensor([[0.1919, 0.4568, 3.2667,  ..., 0.3114, 2.4619, 0.0102],
        [1.2604, 1.7981, 0.1783,  ..., 0.2841, 0.2446, 1.8779],
        [1.3300, 2.0939, 0.1710,  ..., 0.7000, 0.3127, 2.0590]])

As the bag has different sizez, in order to have the same shape (maximum 7) we will pad each tensor. We will check each instance and the shape of the tensor, then we will pad 7-n to the existing tensor. Here, n is total number of instances in the bag.

In [53]:
def tensor_padding(data, max_number_instance):
    new_data = []
    for bag_index in range(len(data)):
        tensor_size = len(data[bag_index][0])
        pad_size = max_number_instance - tensor_size
        p2d = (0,0, 0, pad_size)
        padded_tensor = nn.functional.pad(data[bag_index][0], p2d, 'constant', 0)
        new_data.append((padded_tensor, data[bag_index][1]))
    return new_data

In [54]:
max_num_instance = 7
train_padded = tensor_padding(train_data, max_num_instance)

TypeError: ignored

In [55]:

test_data = [(bag_test_features[i],bag_test_labels[i]) for i in range(len(bag_test_features))]
test_padded = tensor_padding(test_data, max_num_instance)

KeyError: ignored

In [56]:
def data_loaders(train_data, test_data, train_batch_size, val_batch_size):
    train_data = DataLoader(train_data, batch_size=train_batch_size, shuffle=True)
    val_data = DataLoader(test_data, batch_size=val_batch_size, shuffle=False)
    return train_data, val_data

In [57]:

train_data,valid_data = data_loaders(padded_train, padded_test, 1, 1)

NameError: ignored

In [58]:
#Set the batch sizes
train_batch_size = 1
val_batch_size = 1

Defining a model

In [59]:
#Aggregation functions

class SoftMax_Simple(torch.nn.Module):
    def __init__(self, i, n_inst, dim=0):
        super(SoftMax_Simple, self).__init__()
        self.dim = dim
        self.gate = torch.nn.Softmax(dim=self.dim)      
        self.mdl_transform = nn.Sequential(
                            nn.Linear(i, n_inst),
                            nn.LeakyReLU(),
                            nn.Linear(n_inst, i),
                            nn.LeakyReLU(),
                            )
    def forward(self, xx):
        z = self.mdl_transform(xx)
        if self.dim==0:
            z = z.view((z.shape[0],1)).sum(1)
        elif self.dim==1:
            z = z.view((1, z.shape[1])).sum(0)
        gate_ = self.gate(z)
        res = torch.sum(xx* gate_, self.dim)
        return res, gate_

    
class Attention_SoftMax(torch.nn.Module):
    def __init__(self, in_feature = 3, out_feature = None):
        super(Attention_SoftMax, self).__init__()
        self.other_dim = ''
        if out_feature is None:
            out_feature = in_feature
        self.layer_linear_tr = nn.Linear(in_feature, out_feature)
        self.activation = nn.LeakyReLU()
        self.layer_linear_query = nn.Linear(out_feature, 1)
        
    def forward(self, x):
        k = self.layer_linear_tr(x)
        k = self.activation(k)
        attention_map_raw = self.layer_linear_query(k)[...,0]
        attention_map = nn.Softmax(dim=-1)(attention_map_raw)
        result = torch.einsum(f'{self.other_dim}i,{self.other_dim}ij->{self.other_dim}j', attention_map, x)
        return result, attention_map

In [60]:
class Noisy_And(torch.nn.Module):
    def __init__(self, a=10, dims=[1,2]):
        super(Noisy_And, self).__init__()
        self.a = a
        self.b = torch.nn.Parameter(torch.tensor(0.01))
        self.dims =dims
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        t_mean = torch.mean(x, self.dims, True)
        result = (self.sigmoid(self.a * (t_mean - self.b)) - self.sigmoid(-self.a * self.b)) / (
              self.sigmoid(self.a * (1 - self.b)) - self.sigmoid(-self.a * self.b))
        return result
    


class Neural_Net(torch.nn.Module):

    def __init__(self, n=512,n_mid = 1024,n_out=1, dropout=0.2,scoring = None):
        super(Neural_Net, self).__init__()
        self.linear_1 = torch.nn.Linear(n, n_mid)
        self.non_linearity = torch.nn.LeakyReLU()
        self.linear_2 = torch.nn.Linear(n_mid, n_out)
        self.dropout = torch.nn.Dropout(dropout)
        if scoring:
            self.scoring = scoring
        else:
            self.scoring = torch.nn.Softmax() if n_out>1 else torch.nn.Sigmoid()
        
    def forward(self, x):
        z = self.linear_1(x)
        z = self.non_linearity(z)
        z = self.dropout(z)
        z = self.linear_2(z)
        y_pred = self.scoring(z)
        return y_pred
    

class Logistic_Regression(torch.nn.Module):
    def __init__(self, n=512, n_out=1):
        super(Logistic_Regression, self).__init__()
        self.linear = torch.nn.Linear(n, n_out)
        self.scoring = torch.nn.Softmax() if n_out>1 else torch.nn.Sigmoid()

    def forward(self, x):
        a = self.linear(x)
        y_pred = self.scoring(a)
        return y_pred

    
def regularization_loss(param, reg_factor = 0.005, reg_alpha = 0.5):
    param = [p for p in param if len(p.shape)>1]
    l1_reg = nn.L1Loss()
    l2_reg = nn.MSELoss()
    loss_regularization =0
    for p in params:
        loss_regularization += reg_factor*((1-reg_alpha)*l1_reg(p, target=torch.zeros_like(p))
                       + reg_alpha*l2_reg(p, target=torch.zeros_like(p)))
    return loss_regularization

class MIL_NeuralNet(torch.nn.Module):

    def __init__(self, n=512,n_mid=1024,n_classes=1,dropout=0.1,agg = None,scoring=None):
        super(MIL_NeuralNet, self).__init__()
        self.agg = agg if agg is not None else Attention_SoftMax(n)
        
        if n_mid == 0:
            self.bag_model = Logistic_Regression(n, n_classes)
        else:
            self.bag_model = Neural_Net(n, n_mid, n_classes, dropout=dropout, scoring=scoring)
        
    def forward(self, bag_features, bag_lbls=None):
        bag_feature, bag_a, bag_keys = list(zip(*[list(self.agg(ff.float())) + [idx]
                                                    for idx, ff in (bag_features.items())]))
        bag_a = dict(zip(bag_keys, [a.detach().cpu() for a  in bag_a]))
        bag_feature_stacked = torch.stack(bag_feature)
        y_preds = self.bag_model(bag_feature_stacked)
        return y_preds, bag_a, bag_keys

Helper function

In [61]:
def metric_calculation(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_score(p, r, f1, acc, batch_size):
    # Print the utilities
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

# Train & Test

In [62]:

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr0 = 1e-4

# model:
model_mil = MIL_NeuralNet().to(device)

# params you need to specify:
epochs = 5
train_data, val_data = get_data(train_padded, test_padded, 1, 1)
loss_function = torch.nn.BCELoss(reduction='mean')

optimizer = optim.SGD(model.parameters(), lr=lr0, momentum=0.9)

losses = []
batches = len(train_data)
val_batches = len(val_data)

for epoch in range(epochs):
    loss = 0
    # Display progress
    progress = tqdm(enumerate(train_data), desc="Loss: ", total=batches)


    # Train the model
    model_mil.train()
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        X = X.reshape([1,7*512])
        y = y.type(torch.cuda.FloatTensor)
        model_mil.zero_grad()
        outputs = model(X)
        loss = loss_function(outputs, y) 
        loss.backward()
        optimizer.step()

        # Determining the training quality
        current_loss = loss.item()
        total_loss += current_loss

        # Updating the progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # Releasing an unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    #Validate the model
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    # set model to evaluating (testing)
    model_mil.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            X = X.reshape([1,7*512])
            y = y.type(torch.cuda.FloatTensor)
            op = model_mil(X)
            prediced_class =op.detach().round()
            val_losses += loss_function(op, y)
            
            # Calculate the evaluation metrics
            for ac, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                ac.append(calculate_metric(metric, y.cpu(), prediced_classes.cpu()))
          
    print(f"Epoch: {epoch+1}/{epochs}, Training Loss: {total_loss/batches}, Validation Loss: {val_losses/val_batches}")
    print_score(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)
print(f"Training time: {time.time()-start_ts}s")

NameError: ignored